<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/52030543667/in/dateposted-public/" title="Zoom Background"><img src="https://live.staticflickr.com/65535/52030543667_1ec272fe0c_w.jpg" width="400" height="225" alt="Zoom Background"></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

#### Showing...

How to make pandas read from an SQL database, once we have a connection.

How to build a context manager in pure Python, as a convenience for working with databases.

In [1]:
import pandas as pd
import numpy as np

In [2]:
import sqlite3 as sql  # part of Python Standard Library

Have you encountered the Context Manager pattern?  Dunder ("double underline") `enter` and `exit` (i.e. `__enter__` and `__exit__`) get triggered by entering and exiting a context.

What's a context?  An indented code block identified by the keyword `with`.

Inside a context, you might typically want to have some temporary settings, or perhaps an open file or database connection.  Leaving the context returns the settings to normal and/or closes the file or connection.

When I say:
```python
    
    print("Entering context...")
    with Connector('airports.db') as DB:
        df = pd.read_sql("SELECT * FROM Airports;", con=DB.conn)
        print("Exiting context...")
```

I'm triggering three events:
* the `__init__` of the Connector class, in calling it to initialize it
* the `__enter__` method, triggered by `with`, returns the object to be known as... (your choice of legal Python names, DB in this case)
* the `__exit__` method of DB upon exiting the indented scope (the context)

In [3]:
class Connector:
    
    def __init__(self, conn_name):
        self.cn_name = conn_name
        
    def __enter__(self):
        try:
            self.conn = sql.connect(self.cn_name)
            print("Connection: ", self.conn)
            self.curs = self.conn.cursor()
            # self.list_tables() # optional
        except:
            print("No connection")
            raise

        return self
    
    def lookup(self, table, column, code):
        """
        return the data for column = code condition
        """
        self.curs.execute(f"SELECT * FROM {table} WHERE {column} = ?", (code, ))
        return self.curs.fetchone() # could be None, could be a tuple
    
    def list_tables(self):
        """
        print a listing of all the tables in this db
        https://www.sqlitetutorial.net/sqlite-show-tables/
        """
        self.curs.execute("""SELECT name FROM sqlite_schema  
                            WHERE type ='table' AND name 
                            NOT LIKE 'sqlite_%';
                            """)    
        # loop through whatever table names were found 
        # and filtered and print them out.
        for nm in self.curs.fetchall():
            print(nm)
         
    def __exit__(self, *oops):
        """
        I process exceptions i.e. *oops consists of 
        a 3-tuple, we hope filled with Nones because 
        all went well.  Otherwise, exception info.
        return either True or False to determine if
        __exit__ does or does not raise an exception.
        """
        self.conn.close()
        if oops[0]:
            print("An error occurred")
            return False  # raise exception
        return True       # all good

In [4]:
! ls -o airports.db  # make sure we have the db and that it's not 0 bytes

-rw-r--r--  1 kirbyurner  475136 Aug 18 11:25 airports.db


In [5]:
with Connector("./airports.db") as DB:
    DB.list_tables()
    df = pd.read_sql("SELECT * FROM Airports", con = DB.conn)
    print(DB.lookup("Airports", "iata", "SFO"))
    print(DB.lookup("Airports", "iata", "PDX"))

Connection:  <sqlite3.Connection object at 0x7fab9bb42b70>
('Airports',)
('SFO', 'US', 'San Francisco International Airport', 'NA', 'airport', 37.615215, -122.38988, 'large', 1)
('PDX', 'US', 'Portland International Airport', 'NA', 'airport', 45.588997, -122.5929, 'large', 1)


We see above that all went well. `Airports` got listed as the one table inside `airports.db`, and we were apparently able to create a new DataFrame.

Inside the scope of the context, we have a "connected call" e.g. we can use the DB object returned by `__enter__` to communicate with the sqlite3 engine.

The conventions for talking to an SQL database are summarized in what's called the [DB API](https://peps.python.org/pep-0249/), and it's designed to work across languages.

`DB.conn` holds the connection object, while `DB.curs` holds something called a cursor, the holder of the important `execute` method.

`pd.read_sql` is smart enough to only need a connection, and is capable of getting its own cursor object behind the scenes.

In [6]:
df.head()

,iata,iso,name,continent,type,lat,lon,size,status
0,UTK,MH,Utirik Airport,OC,airport,11.233333,169.86667,small,1
1,FIV,US,Five Finger CG Heliport,NA,heliport,NaN,NaN,None,1
2,FAK,US,False Island Seaplane Base,NA,seaplanes,NaN,NaN,None,1
3,BWS,US,Blaine Municipal Airport,NA,closed,NaN,NaN,None,0
4,WKK,US,Aleknagik / New Airport,NA,airport,59.277780,-158.61111,medium,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6726 entries, 0 to 6725
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   iata       6726 non-null   object 
 1   iso        6726 non-null   object 
 2   name       6247 non-null   object 
 3   continent  6726 non-null   object 
 4   type       6726 non-null   object 
 5   lat        6345 non-null   float64
 6   lon        6345 non-null   float64
 7   size       6546 non-null   object 
 8   status     6726 non-null   int64  
dtypes: float64(2), int64(1), object(6)
memory usage: 473.0+ KB
